In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

import sklearn
from nltk.stem.porter import *
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB

from IPython.display import clear_output

import pandas as pd
import numpy as np
import time
import string
from tqdm import notebook
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings("ignore")

# Laad data in

In [3]:
df = pd.read_csv("uploads.csv").drop("Unnamed: 0", axis=1)

In [4]:
df.tail()

,channel,video_id,title,description,conspiracy,transcript
65679,UCgRvm1yLFoaQKhmaTqXk9SA,tt6yWP2VWOU,"New Zealand's Push for Gun Reform, Explained |...",New Zealand is making some big changes. Their ...,False,New Zealand is making some big changes. [ARDEN...
65680,UCgRvm1yLFoaQKhmaTqXk9SA,t7pmHD8rqOA,The Rise and Fall of Myanmar’s Aung San Suu Ky...,A non-violent freedom fighter? A war crimes ap...,False,A non-violent freedom fighter? War crimes apo...
65681,UCgRvm1yLFoaQKhmaTqXk9SA,K7pzjFOe7X0,Spain's Plan to Quit the Coal Industry Explain...,Climate change is happening. And one industry ...,False,Climate change is happening. And one industry ...
65682,UCgRvm1yLFoaQKhmaTqXk9SA,KpYxFouaTvk,How Costa Rica Avoided Cold War Violence | Now...,Costa Rica has sometimes been called “the Swit...,False,Costa Rica has sometimes been called “the Swi...
65683,UCgRvm1yLFoaQKhmaTqXk9SA,BiulFh5uAyM,Trump’s Historic 'National Emergency’ Explaine...,President Donald Trump has declared a national...,False,What you’re about to see… is a historic moment...


In [5]:
df["conspiracy"].value_counts()

False    43528
True     22156
Name: conspiracy, dtype: int64

# Opschonen

In [6]:
# Balanceer de classes
nc = df[df["conspiracy"] == False].sample(22156)
c = df[df["conspiracy"] == True]

equal = pd.concat([nc, c]).reset_index().drop("index", axis=1)
equal.head()

,channel,video_id,title,description,conspiracy,transcript
0,UCQzyyLXtkLaOH4ROYHrwIMw,iFCHKzswBbQ,Bill Burr - I'm Afraid Of Mushrooms,Bill Burr & Craig Conant talk about being sobe...,False,um that reminds me i got to go i was supposed ...
1,UCEP950zoH0nnP2WXHJcWslw,v1Hppd6DbZ0,SMChill - Brave Old World,Get the Newsletter: https://www.subscribepage....,False,[Music] [Laughter] [Applause] [Music] so [Musi...
2,UCt4byS0vUsmAB_hX4NHNUkA,lNBEzjjqN7E,"""I love that line really,"" Jordan Peterson ""Th...","5:55 The link between Circumcision, Tattoos & ...",False,back to the covenant God tells Abraham I make ...
3,UCo4bDsRZPS_Nyljf7MR1ugg,mpqH9SDy-0o,Maintaining Self-Esteem Against the Odds (U109...,When society or misfortune tells people that t...,False,"Reality, captured in user friendly symbols, an..."
4,UC4OvD2yIbofl9l4dIlqSNMw,MO0zc-ZaV4U,Chicago Mayor Blames Trump for Her Incompetence,Chicago's Mayor Lori Lightfoot hasn't been abl...,False,chicago lori lightfoot is she worse than bill ...


In [ ]:
# Vul lege cellen en lowercase
equal["transcript"] = equal["transcript"].fillna("")
equal["title"] = equal["title"].fillna("")
equal["description"] = equal["description"].fillna("")

equal["transcript"] = equal["transcript"].str.lower()
equal["title"] = equal["title"].str.lower()
equal["description"] = equal["description"].str.lower()


In [8]:
stemmer = PorterStemmer()
tqdm.pandas()
stop_words = set(stopwords.words('english')) | set(string.punctuation)

# Stem titles en verwijder stopwords
equal['stemmed_titles'] = equal['title'].progress_apply(
    lambda title: " ".join([stemmer.stem(word) for word in word_tokenize(title)
               if word not in stop_words]))

equal.drop("title", axis = 1, inplace=True)

  0%|          | 0/44312 [00:00<?, ?it/s]

In [22]:
# Stem descriptions en verwijder stopwords
equal['stemmed_descriptions'] = equal['description'].progress_apply(
    lambda description: " ".join([stemmer.stem(word) for word in word_tokenize(str(description))
               if word not in stop_words]))

equal.drop("description", axis = 1, inplace=True)

  0%|          | 0/44312 [00:00<?, ?it/s]

In [9]:
# Stem transcripts en verwijder stopwords
equal['stemmed_transcripts'] = equal['transcript'].progress_apply(
    lambda transcript: " ".join([stemmer.stem(word) for word in word_tokenize(str(transcript))
                                 if word not in stop_words]))

equal.drop("transcript", axis = 1, inplace=True)

  0%|          | 0/44312 [00:00<?, ?it/s]

In [32]:
# Voeg titel, beschrijving en transcript samen tot één kolom
equal["full_text"] = equal["stemmed_titles"] + " " + equal["stemmed_descriptions"] + " " + equal["stemmed_transcripts"]

equal = equal.drop(["stemmed_titles", "stemmed_descriptions", "stemmed_transcripts"], axis = 1)

In [34]:
# equal.to_csv("equal_videos_stemmed.csv")
# equal.tail()

,channel,video_id,conspiracy,full_text
44307,UC5g3fGfwQ_9qZe5UcigRj_w,BVqW8B2jh1A,True,upcom illuminati predict mid 2020 patreon http...
44308,UC5g3fGfwQ_9qZe5UcigRj_w,GJGIJTAQgE4,True,illuminati someth ... patreon http //www.patre...
44309,UC5g3fGfwQ_9qZe5UcigRj_w,gzqou7XiHwQ,True,truth corona zombi viru watch delet twitter ht...
44310,UC5g3fGfwQ_9qZe5UcigRj_w,Y2L6tUxIuOo,True,illuminati predict 2020 analysi tiktok http //...
44311,UC5g3fGfwQ_9qZe5UcigRj_w,NbA_YTvvTTI,True,meet fortun teller live stream http //www.twit...


In [2]:
equal = pd.read_csv("equal_videos_stemmed.csv").drop("Unnamed: 0", axis = 1)

equal.head()

,channel,video_id,conspiracy,full_text
0,UCQzyyLXtkLaOH4ROYHrwIMw,iFCHKzswBbQ,False,bill burr 'm afraid mushroom bill burr craig c...
1,UCEP950zoH0nnP2WXHJcWslw,v1Hppd6DbZ0,False,smchill brave old world get newslett http //ww...
2,UCt4byS0vUsmAB_hX4NHNUkA,lNBEzjjqN7E,False,`` love line realli '' jordan peterson `` land...
3,UCo4bDsRZPS_Nyljf7MR1ugg,mpqH9SDy-0o,False,maintain self-esteem odd u1098 full video when...
4,UC4OvD2yIbofl9l4dIlqSNMw,MO0zc-ZaV4U,False,chicago mayor blame trump incompet chicago 's ...


# Vectorize

In [4]:
# Vectorize words
v = TfidfVectorizer(max_df=.75, min_df=2)
x = v.fit_transform(equal['full_text'])

x

<44312x141463 sparse matrix of type '<class 'numpy.float64'>'
	with 22267877 stored elements in Compressed Sparse Row format>

In [5]:
x.shape

(44312, 141463)

In [6]:
X = x
Y = equal['conspiracy'].values.ravel()

# 10% of the data will be the test set
X_trainval, X_test, y_trainval, y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.1,\
                                                                                  random_state=0)

# of the remaining 90%, 90% will be training set
X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X_trainval, y_trainval,\
                                                                              train_size=0.9,\
                                                                              random_state=0)

# K-nearest neighbors

In [40]:
measures = {"K":[], "Accuracy":[], "Precision":[],"Recall":[], "F1":[]}

for k in notebook.tqdm(range(1, 11)):
    knn = KNeighborsClassifier(k).fit(X_train, y_train)

    y_pred = knn.predict(X_valid)

    measures["K"].append(k)
    measures["Accuracy"].append(sklearn.metrics.accuracy_score(y_valid, y_pred))
    measures["Precision"].append(sklearn.metrics.precision_score(y_valid, y_pred))
    measures["Recall"].append(sklearn.metrics.recall_score(y_valid, y_pred))
    measures["F1"].append(sklearn.metrics.f1_score(y_valid, y_pred))

df_knn = pd.DataFrame(measures)
df_knn

  0%|          | 0/10 [00:00<?, ?it/s]

,K,Accuracy,Precision,Recall,F1
0,1,0.696590,0.638006,0.937101,0.759156
1,2,0.746740,0.926728,0.546929,0.687886
2,3,0.819960,0.803596,0.856511,0.829210
3,4,0.812939,0.862690,0.753317,0.804302
4,5,0.814193,0.795164,0.856511,0.824698
5,6,0.814694,0.840694,0.785749,0.812294
6,7,0.811184,0.789260,0.859459,0.822865
7,8,0.812688,0.823293,0.805897,0.814502
8,9,0.807422,0.781681,0.863882,0.820728
9,10,0.811434,0.809754,0.824079,0.816853


# Naïve Bayes  
### (Currently cannot function because of required array size)

In [1]:
# X_train_dense = X_train.toarray()

# measures = {"NB":[], "Accuracy":[], "Precision":[],"Recall":[], "F1":[]}

# for i in notebook.tqdm([GaussianNB(), MultinomialNB(), ComplementNB(), BernoulliNB()]):
#     nb = i.fit(X_train_dense, y_train)
#     y_pred = nb.predict(X_valid.toarray())

#     measures["NB"].append(str(i))
#     measures["Accuracy"].append(sklearn.metrics.accuracy_score(y_valid, y_pred))
#     measures["Precision"].append(sklearn.metrics.precision_score(y_valid, y_pred))
#     measures["Recall"].append(sklearn.metrics.recall_score(y_valid, y_pred))
#     measures["F1"].append(sklearn.metrics.f1_score(y_valid, y_pred))
    
# df_nb = pd.DataFrame(measures)
# df_nb

# Neural Network

In [ ]:
measures = {"Activation function":[], "# of hidden layers":[], "Neurons per layer":[],
            "Accuracy":[], "Precision":[],"Recall":[], "F1":[]}

done = 0
left = 4*3*3
start = time.time()

for act in ['identity', 'logistic', 'tanh', 'relu']:
    for layers in [1, 10, 25]:
        for neurons in [1, 10, 50]:
            clear_output(wait=True)
            print(f"[{'#' * done}{' ' * left}] {done}/({done}+{left})")
            current = time.time()
            print(f"""\nHours:   {(current - start) // 3600}\nMinutes: {(current - start) // 60}\nSeconds: {(current - start) % 60}""")
            done += 1
            left -= 1
            
            mlp = MLPClassifier(hidden_layer_sizes=[neurons]*layers, activation=act,\
                    random_state=0).fit(X_train, y_train)

            y_pred = mlp.predict(X_test)
            measures["Activation function"].append(act)
            measures["# of hidden layers"].append(layers)
            measures["Neurons per layer"].append(neurons)
            
            measures["Accuracy"].append(sklearn.metrics.accuracy_score(y_valid, mlp.predict(X_valid)))
            measures["Precision"].append(sklearn.metrics.precision_score(y_valid, mlp.predict(X_valid)))
            measures["Recall"].append(sklearn.metrics.recall_score(y_valid, mlp.predict(X_valid)))
            measures["F1"].append(sklearn.metrics.f1_score(y_valid, mlp.predict(X_valid)))
            
df_nn = pd.DataFrame(measures)
df_nn

[########                            ]

Hours:   0.0
Minutes: 2.0
Seconds: 4.095452785491943


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-bfb57ce9bc69>", line 27, in <module>
    measures["Recall"].append(sklearn.metrics.recall_score(y_valid, mlp.predict(X_valid)))
  File "D:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1789, in recall_score
    zero_division=zero_division)
  File "D:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 1490, in precision_recall_fscore_support
    labels=labels, samplewise=samplewise)
  File "D:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 448, in multilabel_confusion_matrix
    y_true = le.transform(y_true)
  File "D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py", line 273, in transform
    _, y = _encode(y, uniques=self.classes_, encode=T

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\lib\site-packages\zmq\eventloop\zmqstream.py", line 432, in _run_callback
    callback(*args, **kwargs)
  File "D:\ProgramData\Anaconda3\lib\site-packages\tornado\stack_context.py", line 276, in null_wrapper
    return fn(*args, **kwargs)
  File "D:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "D:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "D:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "D:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "D:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 

,Activation function,# of hidden layers,Neurons per layer,Accuracy,Precision,Recall,F1
0,identity,1,1,0.724423,0.728293,0.733661,0.730967
1,identity,1,10,0.721916,0.723888,0.735627,0.729710
2,identity,1,50,0.719157,0.727952,0.717936,0.722909
3,identity,10,1,0.715145,0.713539,0.738084,0.725604
4,identity,10,10,0.718154,0.723833,0.723833,0.723833
5,identity,10,50,0.716901,0.716126,0.737592,0.726701
6,identity,25,1,0.713390,0.724121,0.708108,0.716025
7,identity,25,10,0.717653,0.726457,0.716462,0.721425
8,identity,25,50,0.715898,0.714150,0.739066,0.726395
9,logistic,1,1,0.744233,0.752865,0.742506,0.747650


In [16]:
results.to_csv("")

# Linear Regression

# Logistic Regression

# Support-vector machine